In [1]:
import os
import shutil
from openpyxl import load_workbook
import re

# Function to extract numerical values from the given Excel column
def extract_numerical_values(column_values):
    numerical_values = []
    for value in column_values:
        match = re.search(r'\d+', value)
        if match:
            numerical_values.append(match.group())
    return numerical_values

# Function to recursively search for folders and copy the contents
def copy_folder_contents(src_folder, dest_folder):
    for item in os.listdir(src_folder):
        src_item = os.path.join(src_folder, item)
        dest_item = os.path.join(dest_folder, item)
        
        if os.path.isdir(src_item):
            os.makedirs(dest_item, exist_ok=True)
            copy_folder_contents(src_item, dest_item)
        else:
            shutil.copy2(src_item, dest_item)

#######################################################################
# Gets the user input for Excel file, column index, and directory paths

excel_file_path = input("Path to the Excel file: ")
column_index = 4  # Column index (0-based) for column E
start_row = 2     # Row index (0-based) for starting row
end_row = 81      # Row index (0-based) for ending row
src_directory = input("Source directory path: ")
dest_directory = input("Destination directory path: ")

#######################################################################
# Loads the Excel file and extracts numerical values from the specified column

wb = load_workbook(excel_file_path)
ws = wb.active
column_values = [str(ws.cell(row=row, column=column_index + 1).value) for row in range(start_row, end_row + 1)]
numerical_values = extract_numerical_values(column_values)

#######################################################################
# Recursively searches for folders in the source directory that match the numerical values and copies the contents

for root, dirs, files in os.walk(src_directory):
    for dir_name in dirs:
        for num_value in numerical_values:
            if num_value in dir_name:
                src_subfolder = os.path.join(root, dir_name, 'seg_net')
                dest_subfolder = os.path.join(dest_directory, dir_name, 'seg_net')
                copy_folder_contents(src_subfolder, dest_subfolder)

#######################################################################



